In [ ]:
#!/usr/bin/env python
# coding: utf-8


import numpy as np
# import matplotlib.pyplot as plt
import tensorflow as tf
from keras.models import Sequential, load_model
from keras import backend as K
from keras.layers import Dense
from keras.layers import LSTM
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import math

import time
import datetime as dt
from dateutil.parser import parse
import os
import pandas as pd
import openpyxl
import datetime as dt
import cx_Oracle as cx
import os
import warnings
warnings.filterwarnings('ignore')

ddi = 'C:/Users/junei/SODA/20211209_sugong/23개관측소/pre/'
dir_list = os.listdir(ddi)

stand_date = pd.Timestamp(dt.datetime.now().date())

# Hyperparameters
###
tf.set_random_seed(3333) # currently 3333 is the best and 4444 is the second best seed, 6666 is not bad; 7777 soso;
n_pst = 10
r_trn = 0.5
# LTS parameters
n_lts_itr = 20
###
r_lts = 0.5
min_lv = 1
r_lw = 0.1 # minimum value of normalization interval
r_hg = 1 # maximum value of normalization interval
# Optimization parameters
dec_step = 1000 # decay step
###
n_itr = 100000
###
toler1 = 0.02
###
toler2 = 0.0000001
# Design parameters
ref_int = 10000

def DFToDB(df):
    try:
        LOCATION = r"C:\Users\junei\.conda\envs\ll\instantclient_21_3"
        os.environ["PATH"] = LOCATION + ";" + os.environ["PATH"] #환경변수 등록

        conn = cx.connect("NGW", "NGW", cx.makedsn("idc.sodasys.com", "2008", "DAON"))
        curs = conn.cursor()

        rows = [tuple(i) for i in df.to_records(index=False)]
        up_rows = []
        for j in df.index:
            r_1 = int(rows[j][0])
            r_2 = int(rows[j][1])
            r_3 = str(rows[j][2])
            r_4 = parse(rows[j][3])
            r_5 = int(rows[j][4])
            up_rows.append(tuple([r_1, r_2, r_3, r_4, r_5]))

        curs.executemany("INSERT INTO PM_GD60205 \
                         (OBSR_DTA_ID, RESULT_ID, OBSR_DTA_VALUE, \
                         OBSR_DT, QLITY_ID) \
                         VALUES (:1, :2, :3, :4, :5)", up_rows)
        conn.commit()
        curs.close()
        conn.close()
        print("DB Inserted")
        
    except Exception as e:
        print("데이터를 DB에 업로드하는 데 문제가 있습니다.\n", e)
        
def decode(Y_trn, Y_prd1, Y_tst, Y_prd2, n_trn):
    Y_trn = (Y_trn - r_lw) / (r_hg - r_lw) * (maxv_r - minv_r) + minv_r
    Y_trn = np.squeeze(Y_trn) + np.matmul(_X[n_pst:n_pst + n_trn], _W)
    Y_prd1 = (Y_prd1 - r_lw) / (r_hg - r_lw) * (maxv_r - minv_r) + minv_r
    Y_prd1 = np.squeeze(Y_prd1) + np.matmul(_X[n_pst:n_pst + n_trn], _W)
    Y_tst = (Y_tst - r_lw) / (r_hg - r_lw) * (maxv_r - minv_r) + minv_r
    Y_tst = np.squeeze(Y_tst) + np.matmul(_X[n_pst + n_trn:], _W)
    Y_prd2 = (Y_prd2 - r_lw) / (r_hg - r_lw) * (maxv_r - minv_r) + minv_r
    Y_prd2 = np.squeeze(Y_prd2) + np.matmul(_X[n_pst + n_trn:], _W)
    return Y_trn, Y_prd1, Y_tst, Y_prd2
    
def RNN_model(X):
    cell = tf.nn.rnn_cell.BasicLSTMCell(num_units=n_stat_rnn, activation=tf.nn.tanh)
    rnn_out, _states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)
    out = tf.contrib.layers.fully_connected(rnn_out[:, -1], n_neu, activation_fn=tf.nn.tanh,
                                            weights_initializer=tf.contrib.layers.xavier_initializer())#tf.tanh(tf.add(tf.matmul(rnn_out[:, -1], W_i), b_i))
    out = tf.contrib.layers.fully_connected(out, 1, activation_fn=tf.nn.sigmoid,
                                            weights_initializer=tf.contrib.layers.xavier_initializer())#(tf.add(tf.matmul(out, W_o), b_o))
    return out

def n_array(dat_res_raw, dat_exp_raw, dat_io, sav_res):
    n_dat = np.shape(dat_res_raw)[0] - n_pst
    n_exp = np.shape(dat_exp_raw)[1]
    
    #n_exp = 2
    X_dat = np.zeros([n_dat, n_pst, n_exp + 1])
    Y_dat = np.zeros([n_dat, 1])
    
    for ii in range(n_dat):
        X_dat[ii, :, :-1] = dat_exp_raw[ii + 1: ii + n_pst + 1, :n_exp]
        X_dat[ii, :, -1] = dat_res_raw[ii: ii + n_pst]
        Y_dat[ii] = dat_res_raw[ii + n_pst]
        
    n_trn = int(n_dat * r_trn)
    n_tst = n_dat - n_trn
    X_trn, X_tst = np.array(X_dat[:n_trn]), np.array(X_dat[n_trn:])
    Y_trn, Y_tst = np.array(Y_dat[:n_trn]), np.array(Y_dat[n_trn:])

    X_tst_org = np.array(X_tst)
    X_tst[n_pst:, :, -1] = 0
    Y_out = np.array(dat_io[n_pst: n_trn + n_pst])
    
    # Model
    X = tf.placeholder(tf.float32, [None, n_pst, n_exp + 1])
    
    Y = RNN_model(X)
        
    Y_io = tf.placeholder(dtype=tf.float32, shape=[None, 1])
        
    cost = tf.squeeze(tf.matmul(tf.transpose(Y_io), tf.squared_difference(Y, Y_trn)) / tf.reduce_sum(Y_io))
    glo_step = tf.Variable(0, trainable=False)
    learning_rate = tf.train.exponential_decay(strt_lr, glo_step, dec_step, r_dec, staircase=True)
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost, global_step=glo_step)
    
    # Training
    t_strt = time.time()# starting time of learning
    sav = tf.train.Saver()# saver
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        # Restore trained model

        if sav_res == 0:
            err = 1
        # Iterative training
            for ii in range(n_itr + 1):
                err_save = err
                _, err = sess.run([optimizer, cost], feed_dict={X: X_trn, Y_io: Y_out})
                if np.sqrt(err) <= toler1 or np.abs(err - err_save) <= toler2:
                    break
                if ii % 200 == 0:
                    print('Error at {} iteration is {}.'
                          .format(ii, np.sqrt(err)))
            print('Error at {} iteration is {}.'.format(ii, np.sqrt(err)))
        # Save trained model
            sav.save(sess, sess_save_nam)
        elif sav_res == 1:
            sav.restore(sess, sess_save_nam)
        elif sav_res == -1:
            sav.restore(sess, sess_save_nam)
            for ii in range(n_itr + 1):
                _, err = sess.run([optimizer, cost], feed_dict={X: X_trn, Y_io: Y_out})
                if np.sqrt(err) <= toler1:
                    break
                if ii % 200 == 0:
                    print('Error at {} iteration is {}.'
                          .format(ii, np.sqrt(err)))
            print('Error at {} iteration is {}.'.format(ii, np.sqrt(err)))
            sav.save(sess, sess_save_nam)

        Y_prd1 = sess.run(Y, feed_dict={X: X_trn})
        t_end = time.time()# end time of learning
        print('The elapsed time is {} sec'.format(t_end - t_strt))

    # Testing
        Y_prd2 = np.zeros(np.shape(Y_tst))
        for ii in range(n_tst):
            if ii % ref_int == 0 or ii < n_pst:
                Y_prd2[ii] = Y_tst[ii]
                X_tst[:ii + n_pst + 1] = np.array(X_tst_org[:ii + n_pst + 1])
            else:
                Y_prd2[ii] = sess.run(Y, feed_dict={X: X_tst[ii - n_pst: ii + 1]})[-1]
            for jj in range(1, n_pst + 1):
                if ii + jj >= np.shape(X_tst)[0]:
                    break
                else:
                    X_tst[ii + jj, -jj, -1] = Y_prd2[ii]
                    
    Y_trn, Y_prd1, Y_tst, Y_prd2 = decode(Y_trn, Y_prd1, Y_tst, Y_prd2, n_trn)
    return Y_trn, Y_prd1, Y_tst, Y_prd2


# Normalization (0 ~ 1) of both explanatory and response data
def normalization(dat, r1, r2):
    minv = np.min(dat, axis=0)
    maxv = np.max(dat, axis=0)
    dat = (dat - minv) / (maxv - minv) * (r2 - r1)
    np.nan_to_num(dat, copy=False)
    dat = dat + r1
    return dat, minv, maxv


# Detrending and deseasonalization by LTS LTS에 의한 비계절화(?)와 추세 제거
def lts_preproc(dat, flg_season):
    _Y = dat
    n_raw_dat = np.shape(dat)[0]
    if flg_season == 0:# no detrend and no deseasonalization
        _X = np.ones([n_raw_dat, 1])
    elif flg_season == 1:# only detrend and no deseasonalization
        _X = np.ones([n_raw_dat, 2])
        _X[:, 1] = np.arange(n_raw_dat)
    elif flg_season == 2:# detrend and deseasonalization
        _X = np.ones([n_raw_dat, 6])
        _X[:, 1] = np.arange(n_raw_dat)
        _X[:, 2] = np.sin(2 * np.pi / 365 * np.arange(n_raw_dat)) # one cycle per year
        _X[:, 3] = np.cos(2 * np.pi / 365 * np.arange(n_raw_dat)) # one cycle per year
        _X[:, 4] = np.sin(4 * np.pi / 365 * np.arange(n_raw_dat)) # two cycle per year
        _X[:, 5] = np.cos(4 * np.pi / 365 * np.arange(n_raw_dat)) # two cycle per year
    elif flg_season == 3:  # detrend and deseasonalization
        _X = np.ones([n_raw_dat, 5])
        _X[:, 1] = np.sin(2 * np.pi / 365 * np.arange(n_raw_dat))  # one cycle per year
        _X[:, 2] = np.cos(2 * np.pi / 365 * np.arange(n_raw_dat))  # one cycle per year
        _X[:, 3] = np.sin(4 * np.pi / 365 * np.arange(n_raw_dat))  # two cycle per year
        _X[:, 4] = np.cos(4 * np.pi / 365 * np.arange(n_raw_dat))  # two cycle per year
    # Penrose-moore pseudo-inverse
    n_lst_ref = int(n_raw_dat * r_trn)
    _X0 = _X[:n_lst_ref]
    _Y0 = _Y[:n_lst_ref]
    _W = np.matmul(np.linalg.pinv(np.matmul(_X0.T, _X0)),
                   np.matmul(_X0.T, _Y0))
    n_lts = int(n_lst_ref * r_lts)
    for ii in range(n_lts_itr):
        ord = np.argsort(np.abs(_Y0 - np.matmul(_X0, _W)))
        _X1 = _X0[ord[:n_lts]]
        _Y1 = _Y0[ord[:n_lts]]
        _W = np.matmul(np.linalg.pinv(np.matmul(_X1.T, _X1)),
                       np.matmul(_X1.T, _Y1))

    _Y -= np.matmul(_X, _W) # detrending and deseasonalization

    # Outlier exclusion by 6 sigma
    sig = np.std((np.abs(_Y) - _Y) / 2.)
    for ii in range(n_raw_dat):
        if _Y[ii] <= -6 * sig:# use 6 sigma rule
            _Y[ii] = 0
    return _Y, _X, _W

def process(df1, f, sav_res):
    df1['일강수량(mm)'] = 0.0
    df1.reset_index(drop=True, inplace=True)

    try:
        dat_exp_raw1 = df1[['일강수량(mm)', '평균 상대습도(%)', '평균 현지기압(hPa)', '합계 일조 시간(hr)']].values
    except:
        dat_exp_raw1 = df1[['일강수량(mm)', '평균 상대습도(%)', '평균 현지기압(hPa)', '합계 일조시간(hr)']].values

    dat_res_raw1 = df1['수위'].values

    dat_io = np.ones([np.shape(dat_res_raw1)[0], 1])
    for ii in range(np.shape(dat_res_raw1)[0]):
        if dat_res_raw1[ii] <= min_lv:
            dat_io[ii] = 0

    global _X, _W
    dat_res_raw, _X, _W = lts_preproc(dat_res_raw1, f)
    dat_exp_raw, _, _ = normalization(dat_exp_raw1, r_lw, r_hg)

    global minv_r, maxv_r
    dat_res_raw, minv_r, maxv_r = normalization(dat_res_raw, r_lw, r_hg)
    Y_trn, Y_prd1, Y_tst, Y_prd2 = n_array(dat_res_raw, dat_exp_raw, dat_io, sav_res)
    tf.reset_default_graph()
    
    # Save estimation results
    Y_prd=np.zeros(np.shape(Y_prd1)[0]+np.shape(Y_prd2)[0])
    Y_prd[: np.shape(Y_prd1)[0]] = Y_prd1
    Y_prd[np.shape(Y_prd1)[0]:] = Y_prd2
    tf.reset_default_graph()
    return Y_prd

def mk_temp_year(temp_df):
    temp_df1 = temp_df[(temp_df.index <= pd.Timestamp(str(stand_date.year - 1)+'-01-10'))]
    temp_df.reset_index(drop=False, inplace=True)
    return temp_df

def mk_temp_month(temp_df):
    try:
        temp_df = temp_df[(temp_df.index > (stand_date - pd.Timedelta('365d') - pd.Timedelta('90d')))
                         & (temp_df.index <= (stand_date - pd.Timedelta('365d') + pd.Timedelta('100d')))]
        temp_df['일시'] = temp_df.index + pd.Timedelta('365d')

    except:
        print('except')
        temp_df = temp_df[(temp_df.index > (stand_date - pd.Timedelta('730d') - pd.Timedelta('90d')))
             & (temp_df.index <= (stand_date - pd.Timedelta('730d') + pd.Timedelta('100d')))]
        temp_df['일시'] = temp_df.index + pd.Timedelta('730d')
    temp_df.reset_index(drop=True, inplace=True)
    return temp_df    

def up_data(water_spot, n):
    my_tear = {'02' : 14789, '04' : 14995, '06' : 16218, '07' : 14717, '08' : 14700, '10' : 14687, 
               '11' : 14730, '12' : 16189, '14' : 14764, '22' : 14749, '24' : 16197, '25' : 14758}
    water_spot['OBSR_DTA_ID'] = n
    water_spot['RESULT_ID'] = my_tear['{0:02d}'.format(n)]
    water_spot['수위'] = round(water_spot['수위'], 10)
    water_spot['일시'] = water_spot['일시'].astype(str)
    water_spot['QLITY_ID'] = 1 if len(water_spot) <= 300 else 2
    water_spot = water_spot[['OBSR_DTA_ID', 'RESULT_ID', '수위', '일시', 'QLITY_ID']]
    tf.reset_default_graph()
    display(water_spot)
    # 데이터 업로드
#     return DFToDB(water_spot)

def start_pro(n):
    place = dir_list[n]
    print('='*10, place[:-9], '='*10)
    df = pd.read_csv((ddi + place), parse_dates=['일시'])
    
    global sess_save_nam, out_file_nam
    
    sess_save_nam = f'C:/Users/junei/SODA/20211209_sugong/save/{n}_rate8_Session_save_LSTM'# saved session name <--- need to be changed for a new location!!!
    out_file_nam = f'C:/Users/junei/SODA/20211209_sugong/results/{n}_rate8_LSTM.txt'
    
    df2 = df.copy()

    global strt_lr, r_dec, n_stat_rnn, n_lay, n_neu
    
    if n in [4, 6, 10]:
        f = 3
        strt_lr = 0.00005
        r_dec = 0.95
        n_stat_rnn = 16
        n_lay = 4
    elif n == 12:
        f = 3
        strt_lr = 0.00005
        r_dec = 0.75
        n_stat_rnn = 32
        n_lay = 4
    elif n == 14:
        f = 0
        strt_lr = 0.00005
        r_dec = 0.8
        n_stat_rnn = 32
        n_lay = 2
    elif n == 2:
        f = 0
        strt_lr = 0.00005
        r_dec = 0.95
        n_stat_rnn = 16
        n_lay = 4
    elif n in [7, 8, 11]:
        f = 2
        strt_lr = 0.00005
        r_dec = 0.95
        n_stat_rnn = 16
        n_lay = 4
    elif n == 22:
        f = 0
        strt_lr = 0.00005
        r_dec = 0.95
        n_stat_rnn = 32
        n_lay = 2
        
    n_neu = n_stat_rnn
    
#     if (stand_date.month == 1) & (stand_date.day == 20):
        # 테스트 할 때 오늘 날짜 넣어서 하면 됨
    if stand_date.day == 11:
        sav_res = 1
        temp_df = df2.copy()
        temp_df.set_index('일시', inplace=True)
        temp_df = mk_temp_year(temp_df.copy())
        Y_prd = process(temp_df.copy(), f, sav_res)
        temp_df.reset_index(inplace=True)
        water_spot = pd.concat([temp_df['일시'].iloc[:-10], pd.DataFrame(Y_prd, columns=['수위'])], axis=1)
        up_data(water_spot, n)
    
    elif stand_date.day == 15:
        sav_res = 0
        # 불러오기
        temp_df = df2.copy()
        temp_df.set_index('일시', inplace=True)
        
        Y_prd = process(mk_temp_month(temp_df.copy()), f, sav_res)
        temp_df.reset_index(inplace=True)
        water_spot = pd.concat([temp_df['일시'].iloc[:-10], pd.DataFrame(Y_prd, columns=['수위'])], axis=1)
        up_data(temp_df, n)

#         sav_res = 1
#         temp_df = df2.copy()
#         temp_df.set_index('일시', inplace=True)
        
#         Y_prd = process(mk_temp_month(temp_df.copy()), f, sav_res)
#         temp_df.reset_index(inplace=True)
#         water_spot = pd.concat([temp_df['일시'].iloc[:-10], pd.DataFrame(Y_prd, columns=['수위'])], axis=1)
#         up_data(water_spot, n)
        
def data_split(dataset):
    # split into train and test sets
    train_size = int(len(dataset) * 0.5)
    test_size = len(dataset) - train_size
    train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

    train_X, train_Y = train[:, :-1], train[:, -1]
    test_X, test_Y = test[:, :-1], test[:, -1]
    train_X = train_X.reshape(train_X.shape[0], 1, train_X.shape[1])
    test_X = test_X.reshape(test_X.shape[0], 1, test_X.shape[1])

    return train_X, train_Y, test_X, test_Y

def model_train(train_X, train_Y):
    # create and fit the LSTM network
    callback = [EarlyStopping(monitor='loss', patience=3)]

    model = Sequential()
    model.add(LSTM(16, input_shape=(1, 3)))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.fit(train_X, train_Y, epochs=100, batch_size=15, verbose=2, callbacks=callback)

    model.save(sess_save_nam + '.h5')

    return model
    
for spot in [2, 4, 6, 7, 8, 10, 11, 12, 14, 22]:
    start_pro(spot)
    
for spot in [24, 25]:
    n = spot
    place = dir_list[n]
    print('='*10, place[:-9], '='*10)
    df = pd.read_csv((ddi + place), parse_dates=['일시'])
    
    df = df[['일시', '일강수량(mm)', '수위']]
    df['일강수량(mm)1'] = df['일강수량(mm)'].shift(-1)
    df['수위1'] = df['수위'].shift(-1)
    df['diff'] = df['수위1'] - df['수위']
    temp_df = df.copy()
    df3 = temp_df.copy()
    
    sess_save_nam = f'./save/{n}_rate8_Session_save_LSTM'# saved session name <--- need to be changed for a new location!!!
    out_file_nam = f'./results/{n}_rate8_LSTM.txt'

    global df1
    df1 = df3.copy()

    dataset = df1[['일강수량(mm)', '수위', '일강수량(mm)1', 'diff']]

#     if (stand_date.month == 1) & (stand_date.day == 20):
    # 테스트 할 때 오늘 날짜 넣어서 하면 됨
    if stand_date.day == 11:
        K.clear_session()
        model = load_model(sess_save_nam + '.h5')
        
        df2 = df3.copy()
        df2.set_index('일시', inplace=True)
        tmp = mk_temp_year(df2)
        first_value = tmp.at[0, '수위']
        
        mm = tmp[['일강수량(mm)', '수위', '일강수량(mm)1']].values
        testPredict = model.predict(mm.reshape(mm.shape[0], 1, mm.shape[1]))
        tmp['temp'] = testPredict
        
        tmp['수위'] = (tmp['수위'] + tmp['temp']).shift(1)
        tmp.at[0, '수위'] = first_value
        K.clear_session()
        up_data(tmp, n)
        
# #     elif stand_date.day == 15:
#     if stand_date.day == 10:
#         train_X, train_Y, test_X, test_Y = data_split(dataset.values)
#         K.clear_session()
#         model = model_train(train_X, train_Y)
        
#         df2 = df3.copy()
#         df2.set_index('일시', inplace=True)
#         tmp = mk_temp_month(df2)

#         tmp['일강수량(mm)'] = 0
#         tmp['일강수량(mm)1'] = 0
#         tmp = tmp[['일시', '일강수량(mm)', '수위', '일강수량(mm)1']]   

#         mm = tmp[['일강수량(mm)', '수위', '일강수량(mm)1']].values
#         kk = model.predict(mm.reshape(mm.shape[0], 1, mm.shape[1]))
#         tmp['temp'] = kk

#         tmp['수위'] = (tmp['수위'] + tmp['temp']).shift(1)
#         tmp.at[0, '수위'] = first_value
#         K.clear_session()

#         up_data(tmp, n)

C:\Users\junei\.conda\envs\ll\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\junei\.conda\envs\ll\lib\site-packages\tensorflow\python\framework\dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\junei\.conda\envs\ll\lib\site-packages\tensorflow\python\framework\dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\junei\.conda\envs\ll\lib\site-packages\tensorflow\python\framework\dtypes.py:522: 

========== 02_양구방산_암반 ==========
INFO:tensorflow:Restoring parameters from C:/Users/junei/SODA/20211209_sugong/save/2_rate8_Session_save_LSTM
The elapsed time is 0.0985269546508789 sec


,OBSR_DTA_ID,RESULT_ID,수위,일시,QLITY_ID
0,2,14789,245.248328,2010-01-01,2
1,2,14789,245.187903,2010-01-02,2
2,2,14789,245.217058,2010-01-03,2
3,2,14789,245.136791,2010-01-04,2
4,2,14789,245.073295,2010-01-05,2
...,...,...,...,...,...
4337,2,14789,245.054718,2021-11-16,2
4338,2,14789,245.072399,2021-11-17,2
4339,2,14789,245.014962,2021-11-18,2
4340,2,14789,244.979769,2021-11-19,2


INFO:tensorflow:Restoring parameters from C:/Users/junei/SODA/20211209_sugong/save/2_rate8_Session_save_LSTM
The elapsed time is 0.10220837593078613 sec


,OBSR_DTA_ID,RESULT_ID,수위,일시,QLITY_ID
0,2,14789,245.349972,2021-11-14,1
1,2,14789,245.377155,2021-11-15,1
2,2,14789,245.399215,2021-11-16,1
3,2,14789,245.398551,2021-11-17,1
4,2,14789,245.397322,2021-11-18,1
...,...,...,...,...,...
175,2,14789,245.157841,2022-05-08,1
176,2,14789,245.164744,2022-05-09,1
177,2,14789,245.157938,2022-05-10,1
178,2,14789,245.168501,2022-05-11,1


...

,OBSR_DTA_ID,RESULT_ID,수위,일시,QLITY_ID
0,24,16197,4.060000,2009-01-01,2
1,24,16197,4.046645,2009-01-02,2
2,24,16197,4.032721,2009-01-03,2
3,24,16197,4.048318,2009-01-04,2
4,24,16197,4.039794,2009-01-05,2
...,...,...,...,...,...
4743,24,16197,4.049633,2021-12-27,2
4744,24,16197,4.094553,2021-12-28,2
4745,24,16197,4.271586,2021-12-29,2
4746,24,16197,4.295744,2021-12-30,2


,OBSR_DTA_ID,RESULT_ID,수위,일시,QLITY_ID
0,24,16197,2.820000,2021-11-14,1
1,24,16197,2.819774,2021-11-15,1
2,24,16197,2.999067,2021-11-16,1
3,24,16197,3.138674,2021-11-17,1
4,24,16197,3.328363,2021-11-18,1
...,...,...,...,...,...
185,24,16197,4.522390,2022-05-18,1
186,24,16197,4.260620,2022-05-19,1
187,24,16197,4.300860,2022-05-20,1
188,24,16197,4.280738,2022-05-21,1


========== 25_의령낙서_암반 ==========


,OBSR_DTA_ID,RESULT_ID,수위,일시,QLITY_ID
0,25,14758,1.810000,2009-01-01,2
1,25,14758,1.807309,2009-01-02,2
2,25,14758,3.329136,2009-01-03,2
3,25,14758,3.345460,2009-01-04,2
4,25,14758,3.351114,2009-01-05,2
...,...,...,...,...,...
4743,25,14758,-5.334974,2021-12-27,2
4744,25,14758,-4.127020,2021-12-28,2
4745,25,14758,-3.134123,2021-12-29,2
4746,25,14758,-3.236149,2021-12-30,2


,OBSR_DTA_ID,RESULT_ID,수위,일시,QLITY_ID
0,25,14758,2.790000,2021-11-14,1
1,25,14758,2.644588,2021-11-15,1
2,25,14758,2.866010,2021-11-16,1
3,25,14758,3.029917,2021-11-17,1
4,25,14758,3.058861,2021-11-18,1
...,...,...,...,...,...
185,25,14758,4.365004,2022-05-18,1
186,25,14758,4.239056,2022-05-19,1
187,25,14758,4.171244,2022-05-20,1
188,25,14758,4.209993,2022-05-21,1
